In [13]:
import pandas as pd
import numpy as np
from csv_loader import load_marek_csv
from scipy.sparse import identity

In [9]:
data = pd.read_csv('marek_test_mdp.csv')
p, r, _, _ = load_marek_csv('marek_test_mdp.csv')

561it [00:00, 5425.71it/s]

Loading Marek's CSV


381927it [01:13, 5211.35it/s]


In [10]:
data

,idstate_from,rates_from,cases_from,rates_to,cases_to,idstate_to,probability,idaction,reward
0,1,0.5,0.000000,0.736842,0.000000,301,1.223307e-09,0,0.000000
1,1,0.5,0.000000,0.736842,15.151515,302,3.881806e-10,0,0.000000
2,1,0.5,0.000000,0.815789,0.000000,401,1.405600e-08,0,0.000000
3,1,0.5,0.000000,0.815789,15.151515,402,4.460262e-09,0,0.000000
4,1,0.5,0.000000,0.894737,0.000000,501,1.361733e-07,0,0.000000
...,...,...,...,...,...,...,...,...,...
384378,1952,2.0,772.727273,1.368421,1500.000000,1200,4.448691e-03,1,-972.727273
384379,1952,2.0,772.727273,1.447368,1500.000000,1300,1.342030e-03,1,-972.727273
384380,1952,2.0,772.727273,1.526316,1500.000000,1400,3.413466e-04,1,-972.727273
384381,1952,2.0,772.727273,1.605263,1500.000000,1500,7.320363e-05,1,-972.727273


In [69]:
R = np.array(data.groupby(['idaction']).mean()[['rates_from','cases_from']]).T.reshape(4,1)

In [80]:
index = np.array(list(data.groupby(['idstate_from']).mean().index)) - 1
P = p[index, :][:, index, :]
#R = r[index, :][:, index, :]

In [81]:
PHI = np.array(data.groupby(['idstate_from']).mean()[['rates_from', 'cases_from']])
num_states, num_features = PHI.shape
num_actions = data.idaction.unique().shape[0]

In [82]:
c = np.ones(num_states)
P = np.concatenate((P[:,:,0], P[:,:,1]))
#R_TILDE = np.concatenate((R[:,:,0], R[:,:,1]))
#R_TILDE = np.mean(R_TILDE, axis=1)
R_TILDE = R
#R_TILDE = np.eye(num_features*num_actions)
corners = np.zeros((num_states, num_features))
PHI_HAT = np.concatenate((np.concatenate((PHI, corners),axis=1),np.concatenate((corners, PHI), axis=1)),axis=0)
A = np.concatenate((np.eye(num_states), np.eye(num_states)), axis=0) - 0.9 * P

In [83]:
c.shape, PHI.shape

((1623,), (1623, 2))

In [84]:
A.shape, PHI.shape

((3246, 1623), (1623, 2))

In [85]:
PHI_HAT.shape, R_TILDE.shape

((3246, 4), (4, 1))

In [86]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model('lp')

w_xi = m.addMVar(num_features + R_TILDE.shape[1], lb=float('-inf'))
C = c.T @ PHI
m.setMObjective(None, c=C, constant=0.0, xc=w_xi[0:num_features], sense=GRB.MINIMIZE)

m.addConstr(A @ PHI @ w_xi[0:num_features] >= PHI_HAT @ R_TILDE @ w_xi[num_features:])
m.addConstr(sum(w_xi[num_features:]) == 1)
m.addConstrs(w_xi[i] >= 0 for i in range(num_features, num_features + R_TILDE.shape[1]))

m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3248 rows, 3 columns and 9740 nonzeros
Model fingerprint: 0x745b97ca
Coefficient statistics:
  Matrix range     [4e-04, 8e+05]
  Objective range  [2e+03, 1e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 1 columns
Presolve time: 0.02s
Presolved: 3246 rows, 2 columns, 6492 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s

Solved in 5 iterations and 0.04 seconds
Infeasible or unbounded model


In [77]:
print([v for v in m.pi])

GurobiError: Unable to retrieve attribute 'pi'